# Libraries

In [5]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain.chat_models import init_chat_model

import config

class State(TypedDict):
    messages: Annotated[list, add_messages]

llm = init_chat_model(
    "azure_openai:gpt-4.1-nano",
    azure_deployment=config.Config.AZURE_OPENAI_DEPLOYMENT_NAME
)

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Add Steps

In [7]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_ahcLIwuiTffP38oUIb7EHG7i)
 Call ID: call_ahcLIwuiTffP38oUIb7EHG7i
  Args:
    query: LangGraph
  tavily_search (call_Kg9hkLCgRH1yryutTdlBVKpN)
 Call ID: call_Kg9hkLCgRH1yryutTdlBVKpN
  Args:
    query: LangGraph overview
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph overview", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph", "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. At its core, LangGraph uses the power of graph-based architectu

# Replay

In [9]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

print(to_replay.next)
print(to_replay.config)

Num Messages:  11 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  10 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  9 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  9 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  8 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
---------------------------------------------

In [10]:
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_ahcLIwuiTffP38oUIb7EHG7i)
 Call ID: call_ahcLIwuiTffP38oUIb7EHG7i
  Args:
    query: LangGraph
  tavily_search (call_Kg9hkLCgRH1yryutTdlBVKpN)
 Call ID: call_Kg9hkLCgRH1yryutTdlBVKpN
  Args:
    query: LangGraph overview
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph overview", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph", "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. At its core, LangGraph uses the power of graph-based architectures to model and manage the intricate relationships between various components of an AI agent workflow. LangGraph illuminates the processes within 